<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1">Dependencies</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2">Functions</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-3">Paths</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-4">Main</a></span><ul class="toc-item"><li><span><a href="#Cleanup-INT-data" data-toc-modified-id="Cleanup-INT-data-4.1">Cleanup INT data</a></span></li><li><span><a href="#cleanup-COVID-TCR-data---from-adaptive" data-toc-modified-id="cleanup-COVID-TCR-data---from-adaptive-4.2">cleanup COVID TCR data - from adaptive</a></span></li><li><span><a href="#Cleanup-VDJdb-file---curated-by-Shirin" data-toc-modified-id="Cleanup-VDJdb-file---curated-by-Shirin-4.3">Cleanup VDJdb file - curated by Shirin</a></span></li></ul></li></ul></div>

# Dependencies

# Functions

In [1]:
source("/Users/anabbi/git/ped_CapTCRseq/R/Immune_diversity.R")

# Paths

In [2]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"

gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

In [3]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/"

# Main

## Cleanup INT data

In [4]:
load(file = paste0(datapath,"capTCRseq/TRBfiles.RData"))

In [5]:
head(file_df_dedup_trb)

,file_list,samplename,size
,<chr>,<chr>,<dbl>
264,batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315-05-PBMC-DNA,16011
2,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-01-PBMC-DNA.txt,CHP_350-01-PBMC-DNA,85631
3,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-02-PBMC-DNA.txt,CHP_350-02-PBMC-DNA,143632
4,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-03-PBMC-DNA.txt,CHP_350-03-PBMC-DNA,107190
5,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-04-PBMC-DNA.txt,CHP_350-04-PBMC-DNA,54188
6,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_353-02-PBMC-DNA.txt,CHP_353-02-PBMC-DNA,9062


In [6]:
mylist <- list()

for(i in 1:nrow(file_df_dedup_trb)){
    
    onefle <- read.table(paste0(h4hpath, file_df_dedup_trb$file_list[i]), 
                         sep = "\t", header = TRUE, stringsAsFactors = F)  
    # Remove non-productive
    onefle <- onefle[!grepl("_", onefle$aaSeqCDR3),]
    onefle <- onefle[!grepl("[*]", onefle$aaSeqCDR3),]
    onefle$CDR3b <- onefle$aaSeqCDR3
    #next if the file has no productive cdr3
    if(nrow(onefle) == 0){ next}
    #remove scores
    onefle$TRBV <- gsub("[*]0.*", "", onefle$allVHitsWithScore)
    onefle$TRBJ <- gsub("[*]0.*", "", onefle$allJHitsWithScore)

    onefle$count <- onefle$cloneCount
    onefle$clonefraction <- onefle$cloneFraction
    onefle$nSeqCDR3 <- onefle$nSeqCDR3
    onefle$file <- paste0(h4hpath, file_df_dedup_trb$file_list[i])
    onefle$subject.condition <- file_df_dedup_trb$samplename[i]
    mylist[length(mylist) + 1] <- list(onefle[,c("CDR3b", "TRBV", "TRBJ", "subject.condition", "count", "clonefraction",
                                                "nSeqCDR3", "file")])
}

In [7]:
length(mylist)

[1] 580

In [8]:
gliph_input <- do.call("rbind", mylist)

In [9]:
dim(gliph_input)

[1] 72634     8

In [10]:
head(gliph_input)

,CDR3b,TRBV,TRBJ,subject.condition,count,clonefraction,nSeqCDR3,file
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>
1,CASSFYGREADTQYF,TRBV7-2,TRBJ2-3,CHP_315-05-PBMC-DNA,187,0.19767442,TGTGCCAGCAGCTTCTATGGCAGGGAAGCAGATACGCAGTATTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt
3,CASSLMGGDSTEAFF,TRBV7-3,TRBJ1-1,CHP_315-05-PBMC-DNA,48,0.05073996,TGTGCCAGCAGCTTAATGGGGGGAGATAGCACTGAAGCTTTCTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt
4,CASSLTGTGVEKLFF,TRBV7-3,TRBJ1-4,CHP_315-05-PBMC-DNA,48,0.05073996,TGTGCCAGCAGCTTAACCGGGACAGGGGTTGAAAAACTGTTTTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt
5,CASSQRGQSTEQYF,TRBV14,TRBJ2-7,CHP_315-05-PBMC-DNA,42,0.04439746,TGTGCCAGCAGCCAACGGGGACAGTCAACCGAGCAGTACTTC,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt
6,CASSYQAGVGETQYF,TRBV6-6,TRBJ2-5,CHP_315-05-PBMC-DNA,32,0.03382664,TGTGCCAGCAGTTACCAAGCGGGGGTGGGTGAGACCCAGTACTTC,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt
9,CASSVVGVYNSPLHF,TRBV7-2,TRBJ1-6,CHP_315-05-PBMC-DNA,29,0.03065539,TGTGCCAGCAGCGTAGTAGGGGTCTATAATTCACCCCTCCACTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt


In [11]:
library(data.table)

In [13]:
#bind those with the same cdr3 + TRBV
setDT(gliph_input)

ab <- gliph_input[, .(subject = toString(subject.condition), #bind subjects
                      file = toString(file), #bind filenames
                      count = sum(count)),  #get sum of counts
                  by = c("CDR3b", "TRBV", "TRBJ")] #get duplicates sequences with the same cdr3 + TRBV + TRBJ

gliph_input_ab <- merge(gliph_input, ab, by = c("CDR3b","TRBV","TRBJ")) # merge together

In [14]:
head(gliph_input_ab)

CDR3b,TRBV,TRBJ,subject.condition,count.x,clonefraction,nSeqCDR3,file.x,subject,file.y,count.y
<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>
ADITWRGGQPQHF,TRBV3-2,TRBJ1-5,CHP_387-02_PBMC_DNA,12,0.0017035775,GCCGACATTACCTGGAGAGGGGGTCAGCCCCAGCATTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_387-02_PBMC_DNA.txt,CHP_387-02_PBMC_DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_387-02_PBMC_DNA.txt,12
CAAAGGGEAFF,TRBV28,TRBJ1-1,CHP_396-03_PBMC_DNA,16,0.0033105731,TGTGCCGCAGCGGGGGGAGGTGAAGCTTTCTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_396-03_PBMC_DNA.txt,"CHP_396-03_PBMC_DNA, CHP_396-04-PBMC-DNA","/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_396-03_PBMC_DNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch13_26Aug2020/mixcr/clones/CLONES_TRBCHP_396-04-PBMC-DNA.txt",340
CAAAGGGEAFF,TRBV28,TRBJ1-1,CHP_396-04-PBMC-DNA,324,0.0012697069,TGTGCCGCAGCGGGGGGAGGTGAAGCTTTCTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch13_26Aug2020/mixcr/clones/CLONES_TRBCHP_396-04-PBMC-DNA.txt,"CHP_396-03_PBMC_DNA, CHP_396-04-PBMC-DNA","/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_396-03_PBMC_DNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch13_26Aug2020/mixcr/clones/CLONES_TRBCHP_396-04-PBMC-DNA.txt",340
CAAAGGSGVGYEQYF,TRBV3-1,TRBJ2-7,CHP_393-05-PBMC-DNA,151,0.0012674058,TGTGCCGCCGCCGGGGGCAGCGGGGTGGGCTACGAGCAGTACTTC,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch13_26Aug2020/mixcr/clones/CLONES_TRBCHP_393-05-PBMC-DNA.txt,CHP_393-05-PBMC-DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch13_26Aug2020/mixcr/clones/CLONES_TRBCHP_393-05-PBMC-DNA.txt,151
CAAARGFDEQYF,TRBV10-3,TRBJ2-7,CHP-411-02-PBMC-DNA,20,0.0001310135,TGTGCCGCCGCACGGGGTTTCGACGAGCAGTACTTC,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/remaining_batch_4_28June2021/mixcr/clones/CLONES_TRBCHP-411-02-PBMC-DNA.txt,CHP-411-02-PBMC-DNA,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/remaining_batch_4_28June2021/mixcr/clones/CLONES_TRBCHP-411-02-PBMC-DNA.txt,20
CAACEPEVRDTDTQYF,TRBV28,TRBJ2-3,CHP_338-05-PBMC-DNA,11,0.0026397888,TGTGCCGCCTGCGAACCTGAAGTACGGGACACAGATACGCAGTATTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch2_Aug62019/mixcr/clones/CLONES_TRBCHP_338-05-PBMC-DNA.txt,"CHP_338-05-PBMC-DNA, CHP_338-05-PBMC-DNA","/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch2_Aug62019/mixcr/clones/CLONES_TRBCHP_338-05-PBMC-DNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch2_Aug62019/mixcr/clones/CLONES_TRBCHP_338-05-PBMC-DNA.txt",12


In [15]:
#dedup
gliph_input_ab_dedup <- dplyr::distinct(gliph_input_ab, CDR3b, TRBV, TRBJ, subject, .keep_all= TRUE)

In [16]:
dim(gliph_input_ab)
dim(gliph_input_ab_dedup)

[1] 72634    11

[1] 68956    11

In [28]:
datapath

[1] "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"

In [17]:
#index column incase I want to map gliph_input_dedup_ab_subset back to gliph_input_dedup_ab
gliph_input_ab_dedup$index <- 1:nrow(gliph_input_ab_dedup)

In [18]:
save(gliph_input_ab_dedup, file = paste0(datapath, "capTCRseq/gliph_input_ab_dedup.RData"))

In [19]:
colnames(gliph_input_ab_dedup)

[1] "CDR3b"             "TRBV"              "TRBJ"             
 [4] "subject.condition" "count.x"           "clonefraction"    
 [7] "nSeqCDR3"          "file.x"            "subject"          
[10] "file.y"            "count.y"           "index"

In [20]:
#reorder and subset to input to gliph
gliph_input_ab_dedup_subset <- gliph_input_ab_dedup[, c("CDR3b", "TRBV", "TRBJ", "subject", "count.y", "file.y", "index")]
                                                         

In [21]:
colnames(gliph_input_ab_dedup_subset) <- c("CDR3b", "TRBV", "TRBJ", "subject:condition", "count", "file", "index")

In [22]:
save(gliph_input_ab_dedup_subset, file = paste0(datapath, "capTCRseq/gliph_input_ab_dedup_subset.RData"))

## cleanup COVID TCR data - from adaptive

In [23]:
covidfle <- read.table('/Users/anabbi/Desktop/H4H/projects/COVID-19/tcr-cap/Adaptive/MIRA/ImmuneCODE-MIRA-Release002.1/COVID_specific_TCRB.tsv',
                       sep = "\t", header = F)

In [24]:
dim(covidfle)

[1] 162651      6

In [25]:
#remove unproductive
covidfle <- covidfle[!grepl("[*]", covidfle$V1),]
covidfle <- covidfle[ covidfle$V1 != "unproductive",]

In [26]:
dim(covidfle)

[1] 160130      6

In [27]:
#remove duplicated entries
covidfle_uniq <- unique(covidfle)

In [28]:
dim(covidfle_uniq)

[1] 156033      6

In [31]:
covidfle_uniq$V4 <- NULL

In [32]:
head(covidfle_uniq)

,V1,V2,V3,V5,V6
,<chr>,<chr>,<chr>,<chr>,<int>
1,CASGRHNSPLHF,TCRBV12-03/12-04,TCRBJ01-06,eJL143,1
2,CASRPAGVRNEQFF,TCRBV06-06,TCRBJ02-01,eJL146,1
3,CSASPRTLWGEQFF,TCRBV20-X,TCRBJ02-01,eJL146,1
4,CASSLVTSVKDTQYF,TCRBV05-01,TCRBJ02-03,eJL146,1
5,CASSAIVAVYGYTF,TCRBV07-08,TCRBJ01-02,eJL146,1
6,CSVYSAGELFF,TCRBV29-01,TCRBJ02-02,eJL146,1


In [33]:
colnames(covidfle_uniq) <- c("CDR3b", "TRBV", "TRBJ", "subject:condition", "count")

In [34]:
colnames(covidfle_uniq)

[1] "CDR3b"             "TRBV"              "TRBJ"             
[4] "subject:condition" "count"

In [35]:
#bind those with the same cdr3 + TRBV + TRBJ
setDT(covidfle_uniq)

ab <- covidfle_uniq[, .(subject = toString(`subject:condition`), #bind subjects
                        count = sum(count)),  #get sum of counts
                    by = c("CDR3b", "TRBV", "TRBJ")] #get duplicates sequences with the same cdr3 + TRBV + TRBJ

covidfle_uniq_ab <- merge(covidfle_uniq, ab, by = c("CDR3b","TRBV","TRBJ")) # merge together

In [36]:
dim(covidfle_uniq_ab)

[1] 156033      7

In [37]:
colnames(covidfle_uniq_ab)

[1] "CDR3b"             "TRBV"              "TRBJ"             
[4] "subject:condition" "count.x"           "subject"          
[7] "count.y"

In [38]:
#dedup
covidfle_uniq_ab_dedup <- dplyr::distinct(covidfle_uniq_ab, 
                                          CDR3b, TRBV, TRBJ, subject, .keep_all= TRUE)

In [39]:
dim(covidfle_uniq_ab_dedup)

[1] 148678      7

In [40]:
head(covidfle_uniq_ab_dedup)

CDR3b,TRBV,TRBJ,subject:condition,count.x,subject,count.y
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>
AGVKITNSYEQYF,TCRBV20-X,TCRBJ02-07,eHO134,1,eHO134,1
AKNIQYF,TCRBV27-01,TCRBJ02-04,eQD126,1,eQD126,1
APYASKRAGYDNEQFF,TCRBV06-X,TCRBJ02-01,eHO141,1,eHO141,1
ARLFSGANVLTF,TCRBV07-02,TCRBJ02-06,eXL27,1,eXL27,1
CAAAAPDTQYF,TCRBV06-X,TCRBJ02-03,eOX46,1,eOX46,1
CAAADRMTDTQYF,TCRBV24-01,TCRBJ02-03,eHO130,1,eHO130,1


In [41]:
#replaqce TCRB with TRB
covidfle_uniq_ab_dedup$TRBV <- gsub("TCRB", "TRB", covidfle_uniq_ab_dedup$TRBV)
covidfle_uniq_ab_dedup$TRBJ <- gsub("TCRB", "TRB", covidfle_uniq_ab_dedup$TRBJ)

In [42]:
covidfle_uniq_ab_dedup$index <- 1:nrow(covidfle_uniq_ab_dedup)

In [43]:
head(covidfle_uniq_ab_dedup)

CDR3b,TRBV,TRBJ,subject:condition,count.x,subject,count.y,index
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
AGVKITNSYEQYF,TRBV20-X,TRBJ02-07,eHO134,1,eHO134,1,1
AKNIQYF,TRBV27-01,TRBJ02-04,eQD126,1,eQD126,1,2
APYASKRAGYDNEQFF,TRBV06-X,TRBJ02-01,eHO141,1,eHO141,1,3
ARLFSGANVLTF,TRBV07-02,TRBJ02-06,eXL27,1,eXL27,1,4
CAAAAPDTQYF,TRBV06-X,TRBJ02-03,eOX46,1,eOX46,1,5
CAAADRMTDTQYF,TRBV24-01,TRBJ02-03,eHO130,1,eHO130,1,6


In [44]:
head(covidfle_uniq_ab_dedup[order(covidfle_uniq_ab_dedup$count.y, decreasing = T),])

CDR3b,TRBV,TRBJ,subject:condition,count.x,subject,count.y,index
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
CASSIRSSYEQYF,TRBV19-01,TRBJ02-07,eOX46,1,"eOX46, ePD84, eHO128, eHO132, eHO125, ePD83, eHO136, eLH41, eHO124, ePD86, eLH42, eHO130, ePD76, eQD137, eQD123, eMR20, eEE243, eHH173, eHH174, eQD124, eHH169, eEE228, eXL27, eQD115, eXL36, eMR12, eLH47, eDH113, eEE226, eXL37, eDH96, eXL30",32,39668
CASSQTGGRNQPQHF,TRBV12-X,TRBJ01-05,eLH41,1,"eLH41, eJL164, eXL31, eHO132, ePD86, eAV88, ePD84, eEE243, eXL37, eQD118, eEE224, eXL27, eMR12, eOX56, eLH47, eMR18, eLH46, eQD136, eOX43, eOX54, ePD76",21,93523
CASSLAGAYEQYF,TRBV05-01,TRBJ02-07,eJL143,1,"eJL143, ePD79, eHH175, eLH45, eHO126, eXL31, eLH49, eLH42, eJL157, eHO135, eLH54, eMR14, eLH47, eAV91, eQD109, eQD111, eMR23, eQD123, eMR15",19,42124
CASSFSYEQYF,TRBV12-03/12-04,TRBJ02-07,eEE243,1,"eEE243, eHH175, eXL30, eHO130, eJL149, eAV88, eXL27, eOX54, eEE224, eEE228, eOX49, eEE226, eOX52, eEE240, eOX46, eOX43, eXL31",17,31270
CASSLSAPQETQYF,TRBV27-01,TRBJ02-05,eMR17,1,"eMR17, eJL161, eHO134, eQD124, eMR13, eQD111, eQD121, eHO140, eMR12, eQD125, eLH47, eEE226, eQD126, eQD114, eQD110, eQD120",16,60393
CASSVGETQYF,TRBV09-01,TRBJ02-05,eOX56,1,"eOX56, eEE224, eEE226, eEE228, eOX49, eOX54, eOX52, eXL27, eXL30, eOX43, eOX46, eXL32, eEE240, eAV88, eQD128, eHH175",16,109659


In [46]:
covidfle_uniq_ab_dedup$file <- "COVID_specific_TCRB.tsv"

In [47]:
covidfle_uniq_ab_dedup_subset <- covidfle_uniq_ab_dedup[,c("CDR3b", "TRBV", "TRBJ", "subject", "count.y","file", "index")]

In [48]:
colnames(covidfle_uniq_ab_dedup_subset ) <- c("CDR3b", "TRBV", "TRBJ", "subject:condition", "count", "file", "index")

In [50]:
save(covidfle_uniq_ab_dedup_subset, file = paste0(datapath, "capTCRseq/covidfle_uniq_ab_dedup_subset.RData"))

In [65]:
write.table(covidfle_uniq_ab_dedup_subset, file = '/Users/anabbi/Desktop/H4H/projects/COVID-19/tcr-cap/Adaptive/MIRA/ImmuneCODE-MIRA-Release002.1/COVID_specific_TCRB_clean.tsv',
            sep = "\t", quote = F, row.names = F)

## Cleanup VDJdb file - curated by Shirin

In [57]:
VDJdb_tcr <- read.table(file = paste0(datapath, "CapTCRseq/VDJdb_MinimalScoreConfidence3.tsv"), sep = "\t", header = T)

In [58]:
head(VDJdb_tcr)

,complex.id,Gene,aaSeqCDR3,V,J,MHC.A,MHC.B,MHC.class,Epitope,Epitope.gene,Epitope.species,Reference
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0,TRB,CAISGQAVSTDTQYF,TRBV10-3*01,TRBJ2-3*01,HLA-A*02,NA,MHCI,KLVALGINAV,NS3,HCV,PMID:27252176
2,0,TRB,CASDSNTEAFF,TRBV25-1*01,TRBJ1-1*01,HLA-A*02,NA,MHCI,KLVALGINAV,NS3,HCV,PMID:27252176
3,0,TRB,CASGWGSQPQHF,TRBV7-9*01,TRBJ1-5*01,HLA-A*11:01,NA,MHCI,AVFDRKSDAK,EBNA4,EBV,PMID:23267020
4,0,TRB,CASKGQGARDGYTF,TRBV6-6*01,TRBJ1-2*01,HLA-B*35:01,NA,MHCI,IPSINVHHY,pp65,CMV,PMID:23267020
5,0,TRB,CASKMRGDVYEQYF,TRBV18*01,TRBJ2-7*01,HLA-B*08:01,NA,MHCI,RAKFKQLL,BZLF1,EBV,PMID:23267020
6,0,TRB,CASMGNSAGANVLTF,TRBV10-1*01,TRBJ2-6*01,HLA-A*02,NA,MHCI,NLVPMVATV,pp65,CMV,PMID:9971792
